In [155]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

monthly = pd.read_csv('../Dataset/Boulder_Monthly.csv')
monthly.drop(columns={'Unnamed: 0'}, inplace=True)
monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Energy__kWh_   55 non-null     float64
 1   Month          55 non-null     object 
 2   Minimum T      55 non-null     int64  
 3   Maximum T      55 non-null     int64  
 4   Snow           55 non-null     float64
 5   Precipitation  55 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 2.7+ KB


    Scale the Dataset with MinMaxScaler / One-Hot Encode and Extract the Entire Scaled Dataset

In [156]:
# Define the columns we need to scale and we need to use for One-Hot Encoding
columns_to_scale = ['Energy__kWh_', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation']
categorical_columns = ['Month']

# MinMax scaling for numerical columns and One-hot encoding for categorical columns
scaler = MinMaxScaler()
monthly_scaled = monthly.copy()
monthly_scaled[columns_to_scale] = scaler.fit_transform(monthly[columns_to_scale])

# One-hot encoding for categorical columns
onehot_encoder = OneHotEncoder(sparse=False)
categorical_encoded = onehot_encoder.fit_transform(monthly[categorical_columns])

# Get the feature names from the encoder
encoded_columns = []
for col, values in zip(categorical_columns, onehot_encoder.categories_):
    encoded_columns.extend([f'{col}_{value}' for value in values])

# Create DataFrame with encoded columns
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoded_columns)

# Concatenate the new encoded columns to the original DataFrame
monthly_scaled = pd.concat([monthly_scaled, categorical_encoded_df], axis=1)

# Drop the original categorical columns
monthly_scaled = monthly_scaled.drop(categorical_columns, axis=1)
monthly_scaled.columns

Index(['Energy__kWh_', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation',
       'Month_April', 'Month_August', 'Month_December', 'Month_February',
       'Month_January', 'Month_July', 'Month_June', 'Month_March', 'Month_May',
       'Month_November', 'Month_October', 'Month_September'],
      dtype='object')

    Divided the dataset into training, testing, and validation datasets according to 0.70, 0.20, and 0.10, respectively.

In [157]:
# Split the dataset into training, validation, and testing sets
def split_dataset(df, train_ratio, val_ratio):

    total_size = len(df)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_df = df[:train_size]
    val_df = df[train_size:train_size + val_size]
    test_df = df[train_size + val_size:]

    assert len(train_df) + len(val_df) + len(test_df) == total_size, "Dataset not split correctly."

    print(f'Training split ratio:   {round(len(train_df) / len(df), 3)}')
    print(f'Validation split ratio: {round(len(val_df) / len(df), 3)}')
    print(f'Testing split ratio:    {round(len(test_df) / len(df), 3)}')
    print("\nShapes of the datasets:")
    print(train_df.shape, val_df.shape, test_df.shape)

    return train_df, val_df, test_df

In [158]:
train_monthly_scaled, val_monthly_scaled, test_monthly_scaled = split_dataset(monthly_scaled, train_ratio=0.7, val_ratio=0.2)

Training split ratio:   0.691
Validation split ratio: 0.2
Testing split ratio:    0.109

Shapes of the datasets:
(38, 17) (11, 17) (6, 17)


    Create sequences for the Transformer model

In [159]:
# Reshape the data
def create_sequences(data, sequence_length):
    inputs = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequence = data.iloc[i:i + sequence_length].values
        target = data.iloc[i + sequence_length]['Energy__kWh_']  # Predict the next value
        inputs.append(sequence)
        targets.append(target)

    inputs_array = np.array(inputs)
    targets_array = np.array(targets)
    
    print(f'Dataset split into sequences:')
    print(f'Sequences shape: {inputs_array.shape}')
    print(f'Targets shape: {targets_array.shape}\n')

    return np.array(inputs), np.array(targets)

In [171]:
sequence_length = 3
num_features = len(monthly_scaled.columns)

# Create the training, validation, and test data sequences
train_data_inputs, train_data_targets = create_sequences(train_monthly_scaled, sequence_length)
val_data_inputs, val_data_targets = create_sequences(val_monthly_scaled, sequence_length)
test_data_inputs, test_data_targets = create_sequences(test_monthly_scaled, sequence_length)

Dataset split into sequences:
Sequences shape: (35, 3, 17)
Targets shape: (35,)

Dataset split into sequences:
Sequences shape: (8, 3, 17)
Targets shape: (8,)

Dataset split into sequences:
Sequences shape: (3, 3, 17)
Targets shape: (3,)



In [161]:
# The input Datasets must have this input shape (-1, sequence_length, num_features)
train_data_inputs = train_data_inputs.reshape((-1, sequence_length, num_features))
val_data_inputs = val_data_inputs.reshape((-1, sequence_length, num_features))
test_data_inputs = test_data_inputs.reshape((-1, sequence_length, num_features))

train_data_inputs.shape, val_data_inputs.shape, test_data_inputs.shape

((36, 2, 17), (9, 2, 17), (4, 2, 17))

    Create the Transformer Models

In [162]:
%run "../Code/Transformer.ipynb"

# Define the hyperparameters of the manual model
input_shape = (sequence_length, num_features)
num_heads = 1
d_ff = 64
num_layers = 3
dropout_rate = 0.1
encoder_mask = None
decoder_mask = tf.linalg.band_part(tf.ones((sequence_length, sequence_length)), -1, 0)  # Create a lower triangular mask
decoder_mask = 1 - decoder_mask  # Invert the mask

# Create the transformer model
manul_model = TransformerModel(input_shape, num_heads, d_ff, num_layers, dropout_rate, encoder_mask, decoder_mask)

manul_model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 2, 17)]      0           []                               
                                                                                                  
 multi_head_attention_92 (Multi  (None, None, 17)    1224        ['input_11[0][0]',               
 HeadAttention)                                                   'input_11[0][0]',               
                                                                  'input_11[0][0]']               
                                                                                                  
 dropout_186 (Dropout)          (None, None, 17)     0           ['multi_head_attention_92[0][0]']
                                                                                           

In [163]:
# Create the transformer model
keras_model = keras_transformer_model(input_shape, num_heads, d_ff, num_layers, dropout_rate)

keras_model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 2, 17)]      0           []                               
                                                                                                  
 multi_head_attention_99 (Multi  (None, 2, 17)       159         ['input_12[0][0]',               
 HeadAttention)                                                   'input_12[0][0]']               
                                                                                                  
 dropout_201 (Dropout)          (None, 2, 17)        0           ['multi_head_attention_99[0][0]']
                                                                                                  
 layer_normalization_165 (Layer  (None, 2, 17)       34          ['dropout_201[0][0]']     

    Compile the Models

In [164]:
def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(
        tf.keras.backend.mean(
            tf.keras.backend.square(
                y_pred - y_true
            )
        ) + 1e-9
    )

In [165]:
# Define the learning rate for Adam optimizer
learning_rate = 0.01

# Compile the manual model
manul_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse',  metrics=['mae', 'mse', root_mean_squared_error])

# Compile the keras model
keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse',  metrics=['mae', 'mse', root_mean_squared_error])

    Train the Models

In [166]:
# Define the parameters for training
epochs = 200
batch_size = 32

# Convert the data to float32
train_data_inputs = train_data_inputs.astype('float32')
train_data_targets = train_data_targets.astype('float32')

val_data_inputs = val_data_inputs.astype('float32')
val_data_targets = val_data_targets.astype('float32')

train_data_inputs.shape, train_data_targets.shape, val_data_inputs.shape, val_data_targets.shape

((36, 2, 17), (36,), (9, 2, 17), (9,))

In [167]:
# Train the manual model
manul_model.fit(train_data_inputs, train_data_targets,
          validation_data=(val_data_inputs, val_data_targets),
          epochs=epochs, batch_size=batch_size)

Epoch 1/200


2/2 [==============================] - 5s 962ms/step - loss: 0.5505 - mae: 0.6119 - mse: 0.5505 - root_mean_squared_error: 0.8014 - val_loss: 0.1704 - val_mae: 0.3651 - val_mse: 0.1704 - val_root_mean_squared_error: 0.4128
Epoch 2/200
2/2 [==============================] - 0s 39ms/step - loss: 0.6663 - mae: 0.6601 - mse: 0.6663 - root_mean_squared_error: 0.7608 - val_loss: 0.3692 - val_mae: 0.5355 - val_mse: 0.3692 - val_root_mean_squared_error: 0.6076
Epoch 3/200
2/2 [==============================] - 0s 37ms/step - loss: 0.5490 - mae: 0.6004 - mse: 0.5490 - root_mean_squared_error: 0.7699 - val_loss: 0.0999 - val_mae: 0.2698 - val_mse: 0.0999 - val_root_mean_squared_error: 0.3160
Epoch 4/200
2/2 [==============================] - 0s 38ms/step - loss: 0.1631 - mae: 0.3283 - mse: 0.1631 - root_mean_squared_error: 0.4549 - val_loss: 0.3824 - val_mae: 0.5725 - val_mse: 0.3824 - val_root_mean_squared_error: 0.6184
Epoch 5/200
2/2 [==============================] - 0s 39ms/step - loss: 0.3

In [168]:
# Train the keras model
keras_model.fit(train_data_inputs, train_data_targets,
          validation_data=(val_data_inputs, val_data_targets),
          epochs=epochs, batch_size=batch_size)

Epoch 1/200
2/2 [==============================] - 10s 1s/step - loss: 1.1596 - mae: 1.0084 - mse: 1.1596 - root_mean_squared_error: 0.7943 - val_loss: 0.1874 - val_mae: 0.3944 - val_mse: 0.1874 - val_root_mean_squared_error: 0.4329
Epoch 2/200
2/2 [==============================] - 0s 45ms/step - loss: 0.4549 - mae: 0.5694 - mse: 0.4549 - root_mean_squared_error: 0.6633 - val_loss: 0.1573 - val_mae: 0.3607 - val_mse: 0.1573 - val_root_mean_squared_error: 0.3966
Epoch 3/200
2/2 [==============================] - 0s 45ms/step - loss: 0.4056 - mae: 0.5319 - mse: 0.4056 - root_mean_squared_error: 0.5857 - val_loss: 0.1269 - val_mae: 0.3212 - val_mse: 0.1269 - val_root_mean_squared_error: 0.3562
Epoch 4/200
2/2 [==============================] - 0s 44ms/step - loss: 0.3062 - mae: 0.4546 - mse: 0.3062 - root_mean_squared_error: 0.5669 - val_loss: 0.1006 - val_mae: 0.2800 - val_mse: 0.1006 - val_root_mean_squared_error: 0.3171
Epoch 5/200
2/2 [==============================] - 0s 43ms/step -

    Evaluate the Models

In [169]:
# Evaluate on validation set
val_metrics_manul = manul_model.evaluate(val_data_inputs, val_data_targets, return_dict=True)
val_metrics_keras = keras_model.evaluate(val_data_inputs, val_data_targets, return_dict=True)

# Evaluate on test set
test_metrics_manul = manul_model.evaluate(test_data_inputs, test_data_targets, return_dict=True)
test_metrics_keras = keras_model.evaluate(test_data_inputs, test_data_targets, return_dict=True)

# Extract individual metrics
val_loss_manul, val_mae_manul, val_mse_manul, val_rmse_manul = val_metrics_manul['loss'], val_metrics_manul['mae'], val_metrics_manul['mse'], val_metrics_manul['root_mean_squared_error']
test_loss_manul, test_mae_manul, test_mse_manul, test_rmse_manul = test_metrics_manul['loss'], test_metrics_manul['mae'], test_metrics_manul['mse'], test_metrics_manul['root_mean_squared_error']

val_loss_keras, val_mae_keras, val_mse_keras, val_rmse_keras = val_metrics_keras['loss'], val_metrics_keras['mae'], val_metrics_keras['mse'], val_metrics_keras['root_mean_squared_error']
test_loss_keras, test_mae_keras, test_mse_keras, test_rmse_keras = test_metrics_keras['loss'], test_metrics_keras['mae'], test_metrics_keras['mse'], test_metrics_keras['root_mean_squared_error']

print('\n\nManual Transformer:\n-------------------')
print(f'Validation Loss: {val_loss_manul}, Validation MSE: {val_mse_manul}, Validation MAE: {val_mae_manul}, Validation RMSE: {val_rmse_manul}')
print(f'Test Loss: {test_loss_manul}, Test MSE: {test_mse_manul}, Test MAE: {test_mae_manul}, Test RMSE: {test_rmse_manul}')

print('\nKeras Transformer:\n------------------')
print(f'Validation Loss: {val_loss_keras}, Validation MSE: {val_mse_keras}, Validation MAE: {val_mae_keras}, Validation RMSE: {val_rmse_keras}')
print(f'Test Loss: {test_loss_keras}, Test MSE: {test_mse_keras}, Test MAE: {test_mae_keras}, Test RMSE: {test_rmse_keras}')


1/1 [==============================] - 0s 28ms/step - loss: 0.0306 - mae: 0.1615 - mse: 0.0306 - root_mean_squared_error: 0.1749


Manual Transformer:
-------------------
Validation Loss: 0.05501659959554672, Validation MSE: 0.05501659959554672, Validation MAE: 0.1745423674583435, Validation RMSE: 0.234556183218956
Test Loss: 0.0297355018556118, Test MSE: 0.0297355018556118, Test MAE: 0.16150200366973877, Test RMSE: 0.17243985831737518

Keras Transformer:
------------------
Validation Loss: 0.051432158797979355, Validation MSE: 0.051432158797979355, Validation MAE: 0.17358893156051636, Validation RMSE: 0.22678658366203308
Test Loss: 0.030582088977098465, Test MSE: 0.030582088977098465, Test MAE: 0.16150200366973877, Test RMSE: 0.1748773604631424


In [170]:
# Assuming manul_model.predict returns the predictions
val_predictions_manul = manul_model.predict(val_data_inputs)
test_predictions_manul = manul_model.predict(test_data_inputs)

# Assuming keras_model.predict returns the predictions
val_predictions_keras = keras_model.predict(val_data_inputs)
test_predictions_keras  = keras_model.predict(test_data_inputs)

# Calculate MAE and RMSE for validation set
val_mae_manul = np.mean(np.abs(val_data_targets - val_predictions_manul))
val_rmse_manul = np.sqrt(np.mean(np.square(val_data_targets - val_predictions_manul)))

val_mae_keras  = np.mean(np.abs(val_data_targets - val_predictions_keras ))
val_rmse_keras  = np.sqrt(np.mean(np.square(val_data_targets - val_predictions_keras )))

# Calculate MAE and RMSE for test set
test_mae_manul = np.mean(np.abs(test_data_targets - test_predictions_manul))
test_rmse_manul = np.sqrt(np.mean(np.square(test_data_targets - test_predictions_manul)))

test_mae_keras  = np.mean(np.abs(test_data_targets - test_predictions_keras ))
test_rmse_keras  = np.sqrt(np.mean(np.square(test_data_targets - test_predictions_keras )))


print('\n\nManual Transformer:\n-------------------')
print(f'Validation MAE: {val_mae_manul}')
print(f'Validation RMSE: {val_rmse_manul}')
print(f'\nTest MAE: {test_mae_manul}')
print(f'Test RMSE: {test_rmse_manul}')
print('\n==============================')
print('\nKeras Transformer:\n------------------')
print(f'Validation MAE: {val_mae_keras }')
print(f'Validation RMSE: {val_rmse_keras }')
print(f'\nTest MAE: {test_mae_keras }')
print(f'Test RMSE: {test_rmse_keras }')

1/1 [==============================] - 0s 26ms/step


Manual Transformer:
-------------------
Validation MAE: 0.1745423674583435
Validation RMSE: 0.234556183218956

Test MAE: 0.16150199871259407
Test RMSE: 0.17243983993340525


Keras Transformer:
------------------
Validation MAE: 0.17358893156051636
Validation RMSE: 0.22678659856319427

Test MAE: 0.16150199871259407
Test RMSE: 0.17487734073030523
